In [28]:
import os.path
import pandas as pd
from tqdm.notebook import tqdm
import seaborn as sns
from collections import defaultdict
import numpy as np
from collections.abc import Mapping
from dateutil import parser
import csv
import json

import sys
sys.path.append('/home/ubuntu/PYTHON_SCRIPTS')
import normalize_nodes
import importlib
importlib.reload(normalize_nodes)
mondo_npz = {0:np.load("/home/ubuntu/ARAX/MONDO_NPZ/mondos_to_normalized_drugs_21.npz")["arr_0"]}

In [6]:
mondo_npz[0].shape

(660, 52882)

In [9]:
def getNormalizedName(d_idx,no_dict):
    if d_idx in no_dict:
        if(no_dict[d_idx]==None):
            return d_idx
        else:
            return no_dict[d_idx][0]
    return d_idx

In [3]:
ara_begin = defaultdict(dict)
ara_complete = defaultdict(dict)
ara_results = defaultdict(dict)

mondo_first = defaultdict(lambda: parser.parse("2100-01-01"))
mondo_last = defaultdict(lambda: parser.parse("1970-01-01"))

mondo_set = set()
last_began = []
with open("ara_querying.log") as f:
    for line in tqdm(f):
        line = line.strip()
        if("MONDO:") in line:
            time_str = line[5:28]
            time_obj = parser.parse(time_str)
            mondo = "MONDO:" + line.split("MONDO:")[1].split(",")[0]
            mondo_set.add(mondo)
            #Catch the first appeareance of a MONDO and last in our logs
            

#            print(time_obj, line)
            if("Beginning run" in line):
                #if(time_obj < mondo_first[mondo]):mondo_first[mondo]=time_obj
                ara = line.split("MONDO:")[1].split(",")[1]
                ara_begin[ara][mondo] = (time_obj)
                last_began.append((ara,mondo))
            if("Completed run" in line):
                #if(time_obj > mondo_last[mondo]):mondo_last[mondo]=time_obj
                ara = line.split("MONDO:")[1].split(",")[1]
                status = int(line.split("MONDO:")[1].split(",")[2])
                ara_complete[ara][mondo] = (time_obj,status)
                last_began.remove((ara,mondo))
            if("Number results" in line):
                ara = line.split("MONDO:")[1].split(",")[1]
                num_res = int(line.split("MONDO:")[1].split(",")[2])
                ara_results[ara][mondo] = num_res

rows = []
#mondo = "MONDO:0000062"
#mondo_set = ["MONDO:0001082"]
for mondo in mondo_set:
    for ara in ['bte','spoke','medikanren']:
        if(mondo not in ara_begin[ara]):continue
        begin_time = ara_begin[ara][mondo]
        complete_time,status = ara_complete[ara].get(mondo,[None,-1])
#        complete_time = ara_complete[ara].get(mondo,[None,-1])
        num_results = ara_results[ara].get(mondo,-1)
        if(complete_time!=None):
            run_time = (complete_time-begin_time).total_seconds()
        else:
            run_time = -1
        rows.append([mondo,ara,run_time,status,num_results,num_results>0])
query_df = pd.DataFrame(rows,columns=["mondo",'ara','run_time','status','results','has_results'])
print(query_df.mondo.unique().size)

0it [00:00, ?it/s]

22506


In [40]:
#os.listdir(os.path.join(""))
aras = ["spoke","medikanren","bte"]
for target_ara in aras:
    path_to_tables = os.path.join('ara_tables',target_ara)
    table_files = [x for x in os.listdir(path_to_tables) if x.endswith(".csv")]

In [ ]:
table_files = [x for x in os.listdir(path_to_tables) if x.endswith(".csv")]

In [10]:
def getDrugSetForARA(target_ara):
    path_to_tables = os.path.join('ara_tables',target_ara)
    table_files = [x for x in os.listdir(path_to_tables) if x.endswith(".csv")]
    drug_set = set()
    for fname in tqdm(table_files):
        table_file = os.path.join(path_to_tables,fname)
        with open(table_file) as f:
            #Skip the header
            next(f)
            for line in f:
                drug_idx = line.split(',')[1]
                drug_set.add(drug_idx)
    return drug_set

def getDrugCountsForARA(target_ara):
    path_to_tables = os.path.join('ara_tables',target_ara)
    table_files = [x for x in os.listdir(path_to_tables) if x.endswith(".csv")]
    drug_count = defaultdict(int)
    for fname in tqdm(table_files):
        table_file = os.path.join(path_to_tables,fname)
        with open(table_file) as f:
            #Skip the header
            next(f)
            for line in f:
                drug_idx = line.split(',')[1]
                drug_count[drug_idx]+=1
    return drug_count

def getOverlapCountsForMONDO(mondo_idx):
    aras = ["spoke","medikanren","bte"]
    ara_drug_set = {}
    normalized_ara_drug_set = {}
    drug_set = set()
    for target_ara in aras:
        ara_drug_set[target_ara] = set()
        path_to_tables = os.path.join('ara_tables',target_ara)
        table_files = [x for x in os.listdir(path_to_tables) if (x.endswith(".csv") and mondo_idx in x)]
        if(len(table_files)>0):
            print("Found",mondo_idx, "in",target_ara,len(table_files))
        drug_count = defaultdict(int)
        for fname in tqdm(table_files):
            table_file = os.path.join(path_to_tables,fname)
            with open(table_file) as f:
                #Skip the header
                next(f)
                for line in f:
                    drug_idx = line.split(',')[1]
                    ara_drug_set[target_ara].add(drug_idx)
                    drug_count[drug_idx]+=1
    #normalized drug names
    
    pbar = tqdm()
    no_dict = normalize_nodes.normalize_big_list(list(drug_set),pbar)
    for target_ara in aras:
        normalized_ara_drug_set[target_ara] = set()
        for drug in ara_drug_set[target_ara]:
            no_drug_idx = getNormalizedName(drug,no_dict)
            normalized_ara_drug_set[target_ara].add(no_drug_idx)
    drug_idx_cnt = defaultdict(int)
    for target_ara in aras:
        for no_drug_idx in normalized_ara_drug_set[target_ara]:
            drug_idx_cnt[no_drug_idx]+=1
    return drug_idx_cnt
#    return drug_count
    #df = pd.read_csv(f)
    #break

In [15]:
cnt_d = getOverlapCountsForMONDO("MONDO:0015564")
df = pd.DataFrame(cnt_d.items(),columns=["Drug","Count"])

Found MONDO:0015564 in spoke 1


  0%|          | 0/1 [00:00<?, ?it/s]

Found MONDO:0015564 in medikanren 1


  0%|          | 0/1 [00:00<?, ?it/s]

Found MONDO:0015564 in bte 1


  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [18]:
df.Count.value_counts()

1    684
2      5
Name: Count, dtype: int64

In [16]:
getDrugCountsForARA("spoke")

  0%|          | 0/10451 [00:00<?, ?it/s]

defaultdict(int,
            {'PUBCHEM.COMPOUND:443385': 7,
             'PUBCHEM.COMPOUND:470375': 7,
             'PUBCHEM.COMPOUND:23664709': 26,
             'PUBCHEM.COMPOUND:23668834': 23,
             'CHEMBL.COMPOUND:CHEMBL1200530': 8,
             'PUBCHEM.COMPOUND:36921': 7,
             'PUBCHEM.COMPOUND:5904': 23,
             'DRUGBANK:DB01331': 9,
             'CHEMBL.COMPOUND:CHEMBL2373036': 264,
             'CHEMBL.COMPOUND:CHEMBL2373020': 268,
             'CHEMBL.COMPOUND:CHEMBL2373032': 265,
             'CHEMBL.COMPOUND:CHEMBL2373024': 268,
             'CHEMBL.COMPOUND:CHEMBL2373031': 266,
             'CHEMBL.COMPOUND:CHEMBL2373029': 268,
             'CHEMBL.COMPOUND:CHEMBL1790407': 269,
             'CHEMBL.COMPOUND:CHEMBL2373018': 268,
             'CHEMBL.COMPOUND:CHEMBL2373022': 268,
             'CHEMBL.COMPOUND:CHEMBL2373030': 266,
             'CHEMBL.COMPOUND:CHEMBL2373035': 264,
             'CHEMBL.COMPOUND:CHEMBL1790414': 268,
             'CHEMBL.COM

In [4]:
drug_set = getDrugSetForARA("spoke")
pbar = tqdm()
no_d = normalize_nodes.normalize_big_list(list(drug_set),pbar)

  0%|          | 0/10451 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [ ]:
no_d = {}
for drug in drug_set:
    no_d[drug] = (drug,None)

In [ ]:
import csv
mondo_to_drugs = {}
for fname in tqdm(table_files):
    table_file = os.path.join(path_to_tables,fname)
    mondo = fname.replace(".csv","")
    drug_dict = {}
    with open(table_file) as f:
        d_reader = csv.DictReader(f)
        for d in d_reader:
            score = float(d["Score"])
            if(score==0.0):
                score = 0.0000000001
            drug_dict[d["Drug"]] = score
    mondo_to_drugs[mondo] = drug_dict

In [ ]:
for (x,y) in mondo_to_drugs.items():
    print(x,y)
    break

In [6]:
#list(drug_set)
no_d = d
drug_list = []
for x in no_d:
    if(no_d[x]==None):
        drug_list.append(x)
    else:
        drug_list.append(no_d[x][0])

In [7]:
drug_list.sort()

In [8]:
len(drug_list)

51746

In [11]:
len(set(no_d.values()))

50039

In [1]:
mondo_npz = {0:np.load("/home/ubuntu/ARAX/MONDO_NPZ/mondos_to_normalized_drugs_0.npz")["arr_0"]}

NameError: name 'np' is not defined

In [28]:
arax_drug_curie_to_idx = {}
arax_drug_curie_list = []
with open("/home/ubuntu/ARAX/KGML-xDTD/normalized_drug_idxs.csv") as f:
    for i,line in enumerate(f):
        arax_drug_curie_list.append(line.strip().split(',')[1:3])
        arax_drug_curie_to_idx[line.strip().split(',')[1]] = i
arax_mondo_to_idx = {}
arax_mondo_list = []
with open("/home/ubuntu/ARAX/KGML-xDTD/mondos_from_arax.txt") as f:
    for i,line in enumerate(f):
        arax_mondo_list.append(line.strip())
        arax_mondo_to_idx[line.strip()] = i

In [55]:
x = None
del x

In [5]:
#We only need this as a class so that I can dynamically load in the npz for the MONDOs
#we're currently querying.
class ARAX_NPZ(Mapping):
    def __init__(self):
        self.active_npz_idx = None
        self.active_npz = 1
    def __getitem__(self,npz_idx):
        if(self.active_npz_idx!=npz_idx):
            del self.active_npz
            self.active_npz_idx = npz_idx
            self.active_npz = np.load(f"/home/ubuntu/ARAX/MONDO_NPZ/mondos_to_normalized_drugs_{npz_idx}.npz")["arr_0"]
        return self.active_npz
    def __iter__(self):
        return iter(range(0,22))
    def __len__(self):
        return 22
    
#This class builds the first layer of the dictonary. So ARAX_DICT["MONDO:1234567"] builds this.
class ARAX_MONDO_Map(Mapping):
    def __init__(self):
        self.arax_npz = ARAX_NPZ()
    def __getitem__(self, key):
        return ARAXDrugMap(key,self.arax_npz)
        #return "abc"
    def __iter__(self):
        return iter(arax_mondo_list)
    def __len__(self):
        return len(arax_mondo_to_idx)

#This class builds the second layer of the dictonary. So if x=ARAX_DICT["MONDO:1234567"],
# then when you call x["PUBCHEM:8910"], we are using this code.
class ARAX_Drug_Map(Mapping):
    def __init__(self, mondo_idx=0, arax_npz=None):
        if(type(mondo_idx)!=int):
            if(mondo_idx in arax_mondo_to_idx):
                mondo_idx = arax_mondo_to_idx[mondo_idx]
            else: raise ValueError
        self.mondo_idx = mondo_idx
        self.mondo_bin = mondo_idx // 1000
        self.mondo_sub_idx = mondo_idx % 1000
        self.arax_npz = arax_npz
    def __getitem__(self, key):
        if(type(key)!=int):
            key = arax_drug_curie_to_idx[key]
#        if(key in arax_mondo_to_idx):)
        return self.arax_npz[self.mondo_bin][self.mondo_sub_idx][key]
        #return "abc"
    def __iter__(self):
        return iter(arax_drug_curie_list)
    def __len__(self):
        return len(arax_drug_curie_to_idx)
#arax_npzs = 
#arax_dict = ARAXMONDOMap()
#print(arax_dict["MONDO:0000004"]["MESH:D000316"])

In [51]:
for i,k in enumerate(x["MONDO:0000004"]):
    print(k)
    if(i>5):break
    

PUBCHEM.COMPOUND:91852249
MESH:D009183
MESH:D014118
MESH:D000890
MESH:D000977
MESH:D000806
MESH:D000928


In [42]:
mondo_npz[0][0][-55]
#1533 % 1000

0.10502661802569833

In [ ]:
import py2neo
drug_set = set(drug_list)
query = "MATCH (c:`biolink:ChemicalEntity`) RETURN c.equivalent_identifiers, c.status"
graph = py2neo.Graph(host="robokopkg.renci.org")
chem_to_dc = {}
chem_to_status = {}
for m in tqdm(graph.run(query),total=492283):
    m['c.status']
    drug_idxs = m['c.equivalent_identifiers']
    for x in drug_idxs:
        if(x in drug_set): 
            dc_id = [x for x in drug_idxs if "DrugCentral" in x]
            if(len(dc_id)!=0):chem_to_dc[x] = dc_id[0]
            chem_to_status[x] = m['c.status']

In [ ]:
len(chem_to_status)
len(chem_to_dc)

In [6]:
def buildMondoDictDC(chem_to_dc,getMax=True):
    mondo_to_drugs = {}
    for fname in tqdm(table_files):
        table_file = os.path.join(path_to_tables,fname)
        mondo = fname.replace(".csv","")
        drug_dict = {}
        with open(table_file) as f:
            d_reader = csv.DictReader(f)
            for d in d_reader:
                score = float(d["Score"])
                if(score==0.0):
                    score = 0.0000000001
                drug_name = d["Drug"]
                if(drug_name in chem_to_dc):
                    dc_drug_name = chem_to_dc[drug_name]
                    if(getMax):
                        cur_score = drug_dict.get(dc_drug_name,0)
                        if(score > cur_score):drug_dict[dc_drug_name] = score
                    else:
                        drug_dict[dc_drug_name] = score
        mondo_to_drugs[mondo] = drug_dict
    return mondo_to_drugs

In [ ]:
rows = []
heatmap_mondo_list = [fname.replace(".csv","") for fname in table_files]
heatmap_drug_list = sorted(list(chem_to_dc.keys()))
dc_mondo_to_drugs = buildMondoDictDC(chem_to_dc)


In [61]:
def buildHeatmapTable(heatmap_mondo_list,heatmap_drug_list,mondo_to_drugs):
    rows = []
    for mondo in tqdm(heatmap_mondo_list):
        row = [0] * len(heatmap_drug_list)
        if(mondo in mondo_to_drugs):
            score_dict = mondo_to_drugs[mondo]
            for (i,drug) in enumerate(heatmap_drug_list):
                if(drug in score_dict):
                    row[i] = score_dict[drug]
        row = [mondo] + row
        rows.append(row)
    col_names = ["MONDO"] + heatmap_drug_list
    #.reset_index("MONDO")
    return pd.DataFrame(rows,columns=col_names).set_index("MONDO")

In [ ]:
buildHeatmapTable(heatmap_mondo_list,heatmap_drug_list)

In [ ]:
dc_heatmap_drug_list = sorted(list(chem_to_dc.values()))
df = buildHeatmapTable(heatmap_mondo_list,dc_heatmap_drug_list,dc_mondo_to_drugs)

In [ ]:
dc_mondo_to_drugs
chosen_heatmap = "Purples"

In [ ]:
sns.heatmap(df,cmap=chosen_heatmap)

In [64]:
arax_df = buildHeatmapTable(arax_mondo_list,arax_drug_curie_list,ARAXMONDOMap())

  0%|          | 0/21660 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [3]:
arax_npz = ARAX_NPZ()
#rows = []
arax_np = np.vstack((arax_npz[0]))
for i in range(1,22):
    print(arax_np.shape)
    arax_np = np.vstack((arax_np, arax_npz[i]))
df0 = pd.DataFrame(arax_np,columns=arax_drug_curie_list)
del arax_np

(1000, 52882)


KeyboardInterrupt: 

In [71]:
npz0.shape

(2000, 52882)

In [72]:
df0

,PUBCHEM.COMPOUND:91852249,MESH:D009183,MESH:D014118,MESH:D000890,MESH:D000977,MESH:D000806,MESH:D000928,MESH:D011619,MESH:D014149,MESH:D014151,...,UMLS:C5229165,UMLS:C5229166,UMLS:C5229167,UMLS:C5229215,UMLS:C5229216,UMLS:C5229217,UMLS:C5229218,UMLS:C5229219,UMLS:C5229220,UMLS:C5229221
0,0.138631,0.246344,0.353156,0.137719,0.226349,0.578808,0.209298,0.172221,0.424818,0.234673,...,0.036102,0.035646,0.035646,0.028250,0.028250,0.028250,0.028250,0.028250,0.028250,0.028250
1,0.070002,0.086501,0.075001,0.047004,0.057004,0.048000,0.051001,0.049001,0.059501,0.050501,...,0.019584,0.025081,0.025081,0.028054,0.028054,0.028054,0.028054,0.028054,0.028054,0.028054
2,0.157014,0.305222,0.467414,0.485189,0.484641,0.731901,0.523501,0.505672,0.584237,0.539196,...,0.044103,0.046122,0.046122,0.031636,0.031636,0.031636,0.031636,0.031636,0.031636,0.031636
3,0.075003,0.101500,0.100000,0.081989,0.088484,0.093499,0.085978,0.085978,0.093978,0.081478,...,0.029535,0.025585,0.025585,0.036053,0.036053,0.036053,0.036053,0.036053,0.036053,0.036053
4,0.098079,0.155007,0.165514,0.177964,0.182452,0.202489,0.161024,0.164038,0.189849,0.164603,...,0.024810,0.021607,0.021607,0.031556,0.031556,0.031556,0.031556,0.031556,0.031556,0.031556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.141007,0.217500,0.253947,0.256421,0.261932,0.323835,0.270940,0.270447,0.264349,0.257941,...,0.047152,0.053760,0.053760,0.038156,0.038156,0.038156,0.038156,0.038156,0.038156,0.038156
1996,0.118006,0.145000,0.153500,0.131004,0.139000,0.165000,0.131500,0.133000,0.129500,0.123500,...,0.033578,0.036614,0.036614,0.034576,0.034576,0.034576,0.034576,0.034576,0.034576,0.034576
1997,0.169502,0.298533,0.436549,0.519532,0.545958,0.678281,0.572208,0.524817,0.604505,0.598481,...,0.043160,0.055637,0.055637,0.038284,0.038284,0.038284,0.038284,0.038284,0.038284,0.038284
1998,0.153503,0.249236,0.352278,0.450366,0.445968,0.570664,0.556148,0.540721,0.515326,0.516083,...,0.042091,0.046136,0.046136,0.038149,0.038149,0.038149,0.038149,0.038149,0.038149,0.038149


In [12]:
len(arax_mondo_list)
arax_npz = ARAX_NPZ()

In [13]:
z = np.zeros((21660,52882))
for i in range(0,21):
    print(i)
    z[i*1000:(i+1)*1000] = arax_npz[i]
z[21000:21660] = arax_npz[21]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [14]:
df0 = pd.DataFrame(z,columns=arax_drug_curie_list)

In [ ]:
f

In [21]:
#arax_drug_curie_list
#arax_mondo_list

In [22]:
d = {}
with open("/home/ubuntu/ARAX/arax_top_1000_hits.csv") as f:
    for line in f:
        line = line.strip()
        x,y,val = line.split(',')
        x = int(x)
        y = int(y)
        val = float(val)
        d[val] = (x,y)

In [36]:
import csv
with open("arax_top_1000.csv",'w') as f:
    writer = csv.writer(f)
    writer.writerow(["score",'mondo_idx','mondo_name','drug_idx','drug_name'])
#    f
    for val in sorted(d.keys(),reverse=True):
        (x,y) = d[val]
        #y is between 0 and 50,000
        mondo = arax_mondo_list[x]
        mondo_name = mondo_d[mondo][1]
#        mondo_list.append(mondo)
        drug_idx,drug_name = arax_drug_curie_list[y]
        writer.writerow([val,mondo,mondo_name,drug_idx,drug_name])


In [196]:
#mondo_d = normalize_nodes.normalize_big_list(list(mondo_list))
query_df[query_df.mondo=="MONDO:0002142"]

,mondo,ara,run_time,status,results,has_results
14265,MONDO:0002142,bte,146.680,200,500,True
14266,MONDO:0002142,spoke,0.068,200,0,False
14267,MONDO:0002142,medikanren,216.295,200,240,True


In [209]:


#for i in rowGeneratorForFile("bte","MONDO:0002142"):
#sarcoma_df = normDrugDisease(pd.DataFrame(rowGeneratorForMONDO("MONDO:0002142"),columns=['ara','Disease','Drug','Score']))
getAllMONDOsWithWord("pleomorphic sarcoma")
#for i in get500ARAXForMONDO("MONDO:0002142"):
#    print(i)
#score = get500ARAXForMONDO("MONDO:0002142")

undifferentiated high grade pleomorphic sarcoma of bone
undifferentiated pleomorphic sarcoma
cutaneous undifferentiated pleomorphic sarcoma
undifferentiated pleomorphic sarcoma


In [221]:
for i,j in getAllMONDOsWithWord("pleomorphic sarcoma"):
    print(i,j)

MONDO:0002618 undifferentiated high grade pleomorphic sarcoma of bone
MONDO:0006480 undifferentiated pleomorphic sarcoma
MONDO:0002141 cutaneous undifferentiated pleomorphic sarcoma
MONDO:0002142 undifferentiated pleomorphic sarcoma


In [216]:
sarcoma_df = normDrugDisease(pd.DataFrame(tableForMONDOWord("pleomorphic sarcoma"),columns=['ara','Disease','Drug','Score']))
sarcoma_df

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


,ara,Disease,Disease_normalized_idx,Disease_normalized_name,Drug,Drug_normalized_idx,Drug_normalized_name,Score
0,bte,MONDO:0002618,MONDO:0002618,undifferentiated high grade pleomorphic sarcom...,PUBCHEM.COMPOUND:5460033,PUBCHEM.COMPOUND:5460033,Cisplatin,0.618263
1,bte,MONDO:0002618,MONDO:0002618,undifferentiated high grade pleomorphic sarcom...,PUBCHEM.COMPOUND:2907,PUBCHEM.COMPOUND:2907,Cyclophosphamide,0.068076
2,bte,MONDO:0002618,MONDO:0002618,undifferentiated high grade pleomorphic sarcom...,PUBCHEM.COMPOUND:126941,PUBCHEM.COMPOUND:126941,Methotrexate,0.068076
3,bte,MONDO:0002618,MONDO:0002618,undifferentiated high grade pleomorphic sarcom...,UMLS:C4529301,UMLS:C0796392,bevacizumab,0.068076
4,bte,MONDO:0002618,MONDO:0002618,undifferentiated high grade pleomorphic sarcom...,PUBCHEM.COMPOUND:60750,PUBCHEM.COMPOUND:60750,Gemcitabine,0.068076
...,...,...,...,...,...,...,...,...
2775,arax,MONDO:0002142,MONDO:0002142,undifferentiated pleomorphic sarcoma,PUBCHEM.COMPOUND:222757,PUBCHEM.COMPOUND:222757,Estradiol benzoate,0.544529
2776,arax,MONDO:0002142,MONDO:0002142,undifferentiated pleomorphic sarcoma,PUBCHEM.COMPOUND:5311181,PUBCHEM.COMPOUND:5311181,Iloprost,0.544523
2777,arax,MONDO:0002142,MONDO:0002142,undifferentiated pleomorphic sarcoma,CHEMBL.COMPOUND:CHEMBL1201071,CHEMBL.COMPOUND:CHEMBL1201071,TETRACYCLINE PHOSPHATE COMPLEX,0.544491
2778,arax,MONDO:0002142,MONDO:0002142,undifferentiated pleomorphic sarcoma,CHEMBL.COMPOUND:CHEMBL2108429,CHEMBL.COMPOUND:CHEMBL2108429,MEPOLIZUMAB,0.544471


In [217]:
sarcoma_df.to_csv("sarcoma_rankings.csv")

In [17]:

def rowGeneratorForFile(ara,mondo,yield_header=False):
    if(".csv" not in mondo):mondo= mondo + '.csv'
    if(ara==None):fname = mondo
    else: fname = os.path.join('ara_tables',ara,mondo)
    if(not os.path.isfile(fname)):return
    with open(fname) as f:
        reader = csv.reader(f)
        header = next(reader)
        if(yield_header): yield ((header[0],header[1],header[2]))
        for row in reader: yield((row[0],row[1],float(row[2])))
            
def rowGeneratorForMONDO(mondo):
    for ara in ['bte','spoke','medikanren']:
        for row in rowGeneratorForFile(ara,mondo,False):
            yield (ara,row[0],row[1],row[2])
    for row in get500ARAXForMONDO(mondo):
        yield ('arax',row[0],row[1],row[2])
    for row in getROBOKOPResultsForMONDO(mondo):
        yield ('robokop',row[0],row[1],row[2])
    
def getAllMONDOsWithWord(word):
    with open("/home/ubuntu/RANKING/mondo_list.csv") as f:
        reader = csv.reader(f)
        header = next(reader)
        word = word.lower()
        for (mondo_idx,mondo_name) in reader:
            if(word in mondo_name.lower()):
                yield (mondo_idx,mondo_name)
#            break
def tableForMONDOWord(word):
    for mondo_idx,mondo_name in getAllMONDOsWithWord(word):
        for row in rowGeneratorForMONDO(mondo_idx): yield row

def get500ARAXForMONDO(mondo):
    target_line = None
    with open("/home/ubuntu/ARAX/arax_as_table.csv") as f:
        header = next(f).strip().split(',')[1:]
        for line in f:            
            for line in f:
                if(mondo in line[0:100]):
                    target_line = line.strip().split(',')[1:]
    if(target_line==None):return
    rows = []
    for (drug,score) in zip(header,target_line):
        if(score!="-1"):
            rows.append((mondo,drug,float(score)))
    rows.sort(key = lambda x: x[2],reverse=True)
    for row in rows:yield(row)

def getROBOKOPResultsForMONDO(mondo_idx):
    mondo_json_file = os.path.join("/home/ubuntu/NCATS/ara_tables/robokop/",mondo_idx + '.json')
    if (not os.path.isfile(mondo_json_file)):return
    with open(mondo_json_file) as f:
        rows = []
        j = json.load(f)
        for result in j['message']['results']:
            score = result['score']
            if(score==None):continue
            else: 
                score = float(score)
                chemical = result['node_bindings']['chemical'][0]['id']
                rows.append((mondo_idx,chemical,score))
    rows.sort(key = lambda x: x[2],reverse=True)
    for row in rows:yield(row)

                        
def allRowGenerator(target_ara):
    #yield "Disease,Drug,Score,Normalized_Score
    path_to_tables = os.path.join('/home/ubuntu/NCATS/ara_tables',target_ara)
    table_files = [x for x in os.listdir(path_to_tables) if x.endswith(".csv")]
    for fname in tqdm(table_files):
        if("MONDO" not in fname):continue
        table_file = os.path.join(path_to_tables,fname)
        with open(table_file) as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader: yield((row[0],row[1],float(row[2])))

def makeAllRowDF(target_ara, normalize = True):
    df =  pd.DataFrame(allRowGenerator(target_ara),columns=["Disease","Drug","Score"])
#    df = normalize_nodes.normalize_data_frame(df,"Disease")
#    df = normalize_nodes.normalize_data_frame(df,"Drug")
    if(normalize):return normDrugDisease(df)
    else: return df

def normDrugDisease(df):
    df = normalize_nodes.normalize_data_frame(df,"Disease")
    df = normalize_nodes.normalize_data_frame(df,"Drug")
    return df

def getNormDF(mondo):
    return normDrugDisease(pd.DataFrame(rowGeneratorForMONDO(mondo),columns=['ara','Disease','Drug','Score']))

In [16]:
list(getROBOKOPResultsForMONDO("MONDO:0007950"))

[]

In [18]:
#mastocytosis_df = normDrugDisease(pd.DataFrame(tableForMONDOWord("mastocytosis"),columns=['ara','Disease','Drug','Score']))
#mastocytosis_df

In [8]:
mastocytosis_df.to_excel("/home/ubuntu/mastocytosis.xlsx")

In [57]:
bte_df = pd.DataFrame(allRowGenerator("bte"),columns=["Disease","Drug","Score"])
#arax_df = pd.DataFrame(allRowGenerator("arax"),columns=["Disease","Drug","Score"])


  0%|          | 0/22324 [00:00<?, ?it/s]

In [75]:
spoke_df = makeAllRowDF("spoke")

  0%|          | 0/10451 [00:00<?, ?it/s]

In [76]:
medi_df = makeAllRowDF("medikanren")

  0%|          | 0/14659 [00:00<?, ?it/s]

In [117]:
def overlapCount(df1,df2,df3,mondo):
    #gr1 = df1.groupby("MONDO")
    drug_set1 = set(df1[df1.Disease==mondo].Drug_normalized_idx)
    drug_set2 = set(df2[df2.Disease==mondo].Drug_normalized_idx)
    drug_set3 = set(df3[df3.Disease==mondo].Drug_normalized_idx)
    #print('1',len(drug_set1),'2',len(drug_set2),'3',len(drug_set3))
    cnts = defaultdict(int)
    for drug in drug_set1:
        cnts[drug]+=1
    for drug in drug_set2:
        cnts[drug]+=1
    for drug in drug_set3:
        cnts[drug]+=1
    return cnts
    #print(drug_set1)

In [118]:
cnts = overlapCount(bte_df,spoke_df,medi_df,"MONDO:0002913")
df = pd.DataFrame(cnts.items(),columns=["Drug","Cnt"])
df.Cnt.unique()

array([1, 2])

In [144]:
mondo_drug_overlap = []
for (i,mondo) in tqdm(enumerate(query_df.mondo.unique()),total=query_df.mondo.unique().size):
#    print(i)
    cnts = overlapCount(bte_df,spoke_df,medi_df,mondo)
    max_val = max(cnts.values())
    mondo_drug_overlap.append((mondo,max_val))


  0%|          | 0/22506 [00:00<?, ?it/s]

ValueError: max() arg is an empty sequence

In [142]:
mondo_drug_overlap

[('MONDO:0000050', 2)]

In [222]:
#medi_df.value_counts("Disease")
cnts = overlapCount(bte_df,spoke_df,medi_df,"MONDO:0002142")
#3 in cnts

In [223]:
cnts

defaultdict(int,
            {'UNII:J7WWH9M8QS': 1,
             'PUBCHEM.COMPOUND:9913814': 1,
             'UMLS:C0909265': 1,
             'PUBCHEM.COMPOUND:7460': 1,
             'PUBCHEM.COMPOUND:2948': 1,
             'PUBCHEM.COMPOUND:160355': 1,
             'PUBCHEM.COMPOUND:11977753': 1,
             'MESH:D011462': 1,
             'PUBCHEM.COMPOUND:5291': 1,
             'PUBCHEM.COMPOUND:160879': 1,
             'PUBCHEM.COMPOUND:54676038': 1,
             'UMLS:C1096917': 1,
             'PUBCHEM.COMPOUND:4817': 1,
             'PUBCHEM.COMPOUND:41322': 1,
             'PUBCHEM.COMPOUND:65576': 1,
             'UMLS:C0530105': 1,
             'PUBCHEM.COMPOUND:25257557': 1,
             'PUBCHEM.COMPOUND:5315263': 1,
             'MESH:D004742': 1,
             'PUBCHEM.COMPOUND:4068248': 1,
             'PUBCHEM.COMPOUND:44146628': 1,
             'UMLS:C1567346': 1,
             'PUBCHEM.COMPOUND:29566': 1,
             'PUBCHEM.COMPOUND:65577': 1,
             'PUBCHEM.

In [139]:
tmp_df = query_df[query_df.has_results].groupby("mondo")['ara'].count().to_frame().reset_index()
tmp_df.columns = ['mondo','aras']
tmp_df[tmp_df.aras==3].mondo

12       MONDO:0000050
97       MONDO:0000261
98       MONDO:0000262
99       MONDO:0000263
100      MONDO:0000265
             ...      
18389    MONDO:0700081
18408    MONDO:0800026
18411    MONDO:0800029
18459    MONDO:8000010
18466    MONDO:8000018
Name: mondo, Length: 2091, dtype: object

In [128]:
query_df

,mondo,ara,run_time,status,results,has_results
0,MONDO:0002621,bte,14.761,200,12,True
1,MONDO:0002621,spoke,0.075,200,0,False
2,MONDO:0002621,medikanren,10.534,200,250,True
3,MONDO:0024612,bte,11.027,200,33,True
4,MONDO:0024612,spoke,0.064,200,-1,False
...,...,...,...,...,...,...
61057,MONDO:0008797,spoke,6.338,200,300,True
61058,MONDO:0030919,bte,121.472,200,500,True
61059,MONDO:0030919,spoke,0.077,200,0,False
61060,MONDO:0012497,bte,91.214,200,294,True


In [3]:
df = pd.DataFrame(rowGeneratorForMONDO("MONDO:0012497"),columns=

In [9]:
sarcoma_df = normDrugDisease(pd.DataFrame(rowGeneratorForMONDO("MONDO:0002142"),columns=['ara','Disease','Drug','Score']))

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [11]:
castleman_df = getNormDF("MONDO:0015564")

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [12]:
df1 = getNormDF("MONDO:0007254")

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [25]:
influ = getNormDF("MONDO:0005812")

KeyError: nan

In [31]:
importlib.reload(normalize_nodes)
mastocytosis = getNormDF("MONDO:0007950")
#XKCD

KeyError: nan

In [20]:
mastocytosis

,ara,Disease,Disease_normalized_idx,Disease_normalized_name,Drug,Drug_normalized_idx,Drug_normalized_name,Score
0,arax,MONDO:0007950,MONDO:0007950,mastocytosis,PUBCHEM.COMPOUND:5865,PUBCHEM.COMPOUND:5865,Prednisone,0.811497
1,arax,MONDO:0007950,MONDO:0007950,mastocytosis,UNII:4F4X42SYQ6,UNII:4F4X42SYQ6,RITUXIMAB,0.804914
2,arax,MONDO:0007950,MONDO:0007950,mastocytosis,PUBCHEM.COMPOUND:6741,PUBCHEM.COMPOUND:6741,Methylprednisolone,0.790177
3,arax,MONDO:0007950,MONDO:0007950,mastocytosis,UMLS:C1268567,UMLS:C1268567,Protein-tyrosine kinase inhibitor,0.763046
4,arax,MONDO:0007950,MONDO:0007950,mastocytosis,PUBCHEM.COMPOUND:5284373,PUBCHEM.COMPOUND:5284373,cyclosporin A,0.762163
...,...,...,...,...,...,...,...,...
776,arax,MONDO:0007950,MONDO:0007950,mastocytosis,CHEMBL.COMPOUND:CHEMBL3833337,CHEMBL.COMPOUND:CHEMBL3833337,DICOBALT EDETATE,0.089005
777,arax,MONDO:0007950,MONDO:0007950,mastocytosis,UMLS:C2348263,UMLS:C2348263,Dietary Copper,0.089003
778,arax,MONDO:0007950,MONDO:0007950,mastocytosis,CHEMBL.COMPOUND:CHEMBL4297226,CHEMBL.COMPOUND:CHEMBL4297226,CANERPATUREV,0.089003
779,arax,MONDO:0007950,MONDO:0007950,mastocytosis,PUBCHEM.COMPOUND:53480856,PUBCHEM.COMPOUND:53480856,1-(1-Enyl-stearoyl)-2-erucoyl-sn-glycero-3-pho...,0.089003


In [14]:
sarcoma_df.to_csv("sarcoma.csv",index=False)

In [15]:
castleman_df.to_csv("castleman.csv",index=False)

In [16]:
df1.to_csv("breast_cancer.csv",index=False)

In [17]:
influ.to_csv("influenza.csv",index=False)

In [18]:
covid_df = getNormDF("MONDO:0100096")
covid_df.to_csv("covid_19.csv",index=False)

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [9]:
def mondo_to_csv(mondo,csv_name):
    covid_df = getNormDF(mondo)
    covid_df.to_csv(csv_name,index=False)

In [37]:
mondo_to_csv("MONDO:0005009","heart_failure.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../RANKING/ROBOKOP-Dev_April_17/MONDO/MONDO:0005009.json'

In [22]:
mondo_to_csv("MONDO:0001657","brain_cancer.csv")

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [41]:
mondo_to_csv("MONDO:0001112",'plague.csv')

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [27]:
mondo_to_csv("MONDO:0005083","psoriasis.csv")

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [31]:
mondo_to_csv("MONDO:0005090","schizophrenia.csv")

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [26]:
mondo_to_csv("MONDO:0005109","hiv.csv")

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [28]:
mondo_to_csv("MONDO:0005130","celiac_disease.csv")

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [29]:
mondo_to_csv("MONDO:0005709","common_cold.csv")

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [12]:
mondo_to_csv("MONDO:0034099","SYNGAP1.csv")

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [14]:
list(getROBOKOPResultsForMONDO())

[('MONDO:0015564', 'PUBCHEM.COMPOUND:460612', 0.8309466158039879),
 ('MONDO:0015564', 'UNII:T4H8FMA7IM', 0.34259468803444293),
 ('MONDO:0015564', 'PUBCHEM.COMPOUND:10150081', 0.1224958624056823),
 ('MONDO:0015564', 'PUBCHEM.COMPOUND:5426', 0.0491681698392424)]

In [15]:
mondo_to_csv("MONDO:0015564","castleman_ranked.csv")

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [16]:
mondo_to_csv("MONDO:0035838","imdb_ranked.csv")

UnboundLocalError: local variable 'target_line' referenced before assignment

In [13]:
top_10_hits = []
#for ara in ['bte','spoke','medikanren']:
def all_gen(ara):
    for mondo in tqdm(getAllMONDOsWithWord(""),total=22506):
        for row in rowGeneratorForFile(ara,mondo,False):
            yield (ara,row[0],row[1],row[2])
#for row in get500ARAXForMONDO(mondo):
#    yield ('arax',row[0],row[1],row[2])
#for row in getROBOKOPResultsForMONDO(mondo):
#    yield ('robokop',row[0],row[1],row[2])

In [12]:
len(list(getAllMONDOsWithWord("")))

22506

In [24]:
bte_df = makeAllRowDF('bte',False)
bte_df.nlargest(10,"Score")

  0%|          | 0/22324 [00:00<?, ?it/s]

,Disease,Drug,Score
3753527,MONDO:0016575,PUBCHEM.COMPOUND:5755,1.0
1514628,MONDO:0019950,PUBCHEM.COMPOUND:5755,1.0
3753528,MONDO:0016575,PUBCHEM.COMPOUND:5280793,1.0
1514629,MONDO:0019950,PUBCHEM.COMPOUND:12035,1.0
46638,MONDO:0012084,UMLS:C5419302,1.0
2716903,MONDO:0019219,UMLS:C5419302,1.0
2860898,MONDO:0019250,UMLS:C5419302,1.0
3158523,MONDO:0017759,UMLS:C5419302,1.0
3753529,MONDO:0016575,PUBCHEM.COMPOUND:12035,1.0
3753530,MONDO:0016575,PUBCHEM.COMPOUND:5281004,1.0


In [35]:
normDrugDisease(bte_df.nlargest(10,"Score")).to_excel("bte_top_10.xlsx")

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [28]:
spoke_df = makeAllRowDF('spoke',False)

  0%|          | 0/10451 [00:00<?, ?it/s]

In [31]:
normDrugDisease(spoke_df.nlargest(10,"Score")).to_excel("spoke_top_10.xlsx")

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)


In [ ]:
medi_df = makeAllRowDF('medikanren')

In [33]:
rk_df = makeAllRowDF('robokop',False)

  0%|          | 0/22320 [00:00<?, ?it/s]

In [34]:
normDrugDisease(rk_df.nlargest(10,"Score")).to_excel("rk_top_10.xlsx")

/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[idx_col] = df.apply(lambdaFunc,index=0, key=key, axis=1)
/home/ubuntu/NCATS/../PYTHON_SCRIPTS/normalize_nodes.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_col] = df.apply(lambdaFunc,index=1, key=key, axis=1)
